In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
#path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
#path = "/data/sunilruf/prune/wanda/out/llama2_7b/model"
path = "/data/sunilruf/llama/phi3/checkpoint-49"
path = "meta-llama/Llama-2-7b-chat-hf"
#path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50/"
#path = "llamaai/llama-7B-Instruct-v0.2"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.device

device(type='cuda', index=0)

In [88]:
for param in model.parameters():
    print(param.shape)
    break

torch.Size([32000, 4096])


In [22]:
for param in model.parameters():
    if param.data.sum() < 2:
        param.requires_grad = False
    

In [ ]:
for param in model.parameters():
    print(param.requires_grad)

In [8]:
for i, param in enumerate(model.parameters()):
    if i == 0:
        print("yes")
        param.requires_grad = False

yes


In [2]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
rouge = evaluate.load('rouge')
import random
random.seed(0)

In [1]:
import json

json_file_path = '../../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import torch
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

In [4]:
val = 0
for j in range(3010,3020):
    val += (len(extes_data[j]['content']))
    
print(val)

164


In [3]:
print(extes_data[0]['content'])

[{'User': 'Hey, are you free? I need someone to talk to.'}, {'AI Strategy': '', 'AI': "Of course! I'm here to listen. What's been going on?"}, {'User': "I had a rough day at work, and then my significant other texted me out of the blue, saying they need some space. I don't know what to do or how to feel about it."}, {'AI Strategy': 'Reflective Statements', 'AI': "That sounds really tough. It's completely normal to feel confused and insecure when a situation like this arises. Have you had any discussions about needing space in the past?"}, {'User': "No, we haven't really talked about this before. It just came as a surprise. I'm worried that they're unhappy in our relationship."}, {'AI Strategy': 'Emotional Validation', 'AI': "I can understand why you would feel that way. It's natural to jump to conclusions and worry about the state of your relationship. Remember, it's crucial not to blame yourself. Relationships go through ups and downs, and communication is key."}, {'User': "You're rig

In [7]:
import torch
from tqdm import tqdm
import numpy as np
import time
start_time = time.time()

max_length = 1024
stride = 128
smoothie = SmoothingFunction().method4
#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024


final_nlls = []

final_nlls_boh = []
prev_end_loc = 0
messages = []
i = 1
final_messages = []
final_messages_boh = []
final_messages_boh1 = []
final_perplexity = []
final_perplexity_boh = []
bleu_final, bleu_boh_final = [], []
rouge_1_final, rouge_2_final, rougle_l_final = [], [], []
Bert_P_final, Bert_R_final, Bert_F1_final = [], [], []
rouge_1_boh_final, rouge_2_boh_final, rougle_l_boh_final = [], [], []
Bert_P_boh_final, Bert_R_boh_final, Bert_F1_boh_final = [], [], []
output_len_final, output_len_boh_final = [], []
for j in range(3010,3020):
    #num = random.choice(range(3000, 3110))
    num = j
    data = extes_data[num]['content']
    print(data)
    perplexity = []
    perplexity_boh = []
    nlls = []
    nlls_boh = []
    #data = data
    messages = []
    messages_boh = []
    messages_boh1 = []
    bleu_1, bleu_1_boh = [], []
    rouge_1, rouge_2, rougle_l = [], [], []
    Bert_P, Bert_R, Bert_F1 = [], [], []
    rouge_1_boh, rouge_2_boh, rougle_l_boh = [], [], []
    Bert_P_boh, Bert_R_boh, Bert_F1_boh = [], [], []
    output_len, output_len_boh = [], []
    try:
        for i in range(0, len(data), 2):
            for begin_loc in (range(0, seq_len, stride)):
                #print(data[i])
                if 'User' not in data[i]:
                    i = i+1
                #print(data[i]['User'])
                #print(data[i+1]['AI'])
                messages.append({"role": "user", "content":  data[i]['User']})
                messages_boh.append({"role": "user", "content":  remove_stopwords(data[i]['User'])})
                messages_boh1.append({"role": "user", "content":  data[i]['User']})
                #i+=1
                end_loc = min(begin_loc + max_length, seq_len)
                trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
                #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
                input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                target_ids = input_ids.clone()
                target_ids_boh = input_ids_boh.clone()
                target_ids[:, :-trg_len] = -100
                target_ids_boh[:, :-trg_len] = -100

                with torch.no_grad():
                    
                    
                    val, _ = generate_content(tokenizer, messages)
                    val_boh, _ = generate_content(tokenizer, messages_boh)
                    candidate = val
                    candidate_boh = val_boh
                    reference = data[i+1]['AI']
                    references = data[i+1]['AI'].split(" ")
                    candidates = val.split(" ")
                    candidates_boh = val_boh.split(" ")
                    outputs = model(input_ids, labels=target_ids)
                    print(val)
                    print(val_boh)
                    messages.append({"role": "assistant", "content": val})
                    print(outputs.logits.shape)
                    output_len.append(outputs.logits.shape[1])
                    outputs_boh = model(input_ids_boh, labels=target_ids_boh)
                    print(outputs_boh.logits.shape)
                    output_len_boh.append(outputs_boh.logits.shape[1])
                    messages_boh.append({"role": "assistant", "content": remove_stopwords(val_boh)})
                    messages_boh1.append({"role": "assistant", "content": val_boh})
                        
                    
                    neg_log_likelihood = outputs.loss
                    neg_log_likelihood_boh = outputs_boh.loss
                    
                    bleu_1.append(sentence_bleu(references, candidates, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results = rouge.compute(predictions=[candidate], references=[reference])
                    rouge_1.append(results['rouge1'])
                    rouge_2.append(results['rouge2'])
                    rougle_l.append(results['rougeL'])
                    
                    P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-base-uncased')
                    Bert_P.append(P)
                    Bert_R.append(R)
                    Bert_F1.append(F1)
                    
                    bleu_1_boh.append(sentence_bleu(references, candidates_boh, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results_boh = rouge.compute(predictions=[candidate_boh], references=[reference])
                    rouge_1_boh.append(results_boh['rouge1'])
                    rouge_2_boh.append(results_boh['rouge2'])
                    rougle_l_boh.append(results_boh['rougeL'])
                    
                    P, R, F1 = score([candidate_boh], [reference], lang='en', model_type='bert-base-uncased')
                    Bert_P_boh.append(P)
                    Bert_R_boh.append(R)
                    Bert_F1_boh.append(F1)
                    #print(neg_log_likelihood.shape)

                nlls.append(neg_log_likelihood)
                nlls_boh.append(neg_log_likelihood_boh)
                #print(len(nlls))
                prev_end_loc = end_loc
                if end_loc == seq_len:
                    break

            ppl = torch.exp(torch.stack(nlls).mean())
            ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
            print(ppl)
            print(ppl_boh)
            perplexity.append(ppl)
            perplexity_boh.append(ppl_boh)
            
    except Exception as e:
        print(e)
    
    final_perplexity.append(perplexity)
    final_perplexity_boh.append(perplexity_boh)
    final_nlls.append(nlls)
    final_nlls_boh.append(nlls_boh)
    final_messages.append(messages)
    final_messages_boh.append(messages_boh)
    final_messages_boh1.append(messages_boh1)
    bleu_final.append(bleu_1)
    rouge_1_final.append(rouge_1)
    rouge_2_final.append(rouge_2)
    rougle_l_final.append(rougle_l)
    Bert_P_final.append(Bert_P)
    Bert_R_final.append(Bert_R)
    Bert_F1_final.append(Bert_F1)
    bleu_boh_final.append(bleu_1_boh)
    rouge_1_boh_final.append(rouge_1_boh)
    rouge_2_boh_final.append(rouge_2_boh)
    rougle_l_boh_final.append(rougle_l_boh)
    Bert_P_boh_final.append(Bert_P_boh)
    Bert_R_boh_final.append(Bert_R_boh)
    Bert_F1_boh_final.append(Bert_F1_boh)
    output_len_final.append(output_len)
    output_len_boh_final.append(output_len_boh)
    
end_time = time.time()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'User': 'Hey, do you have a minute? Can I talk to you about something?'}, {'AI Strategy': 'Reflective Statements', 'AI': "Of course! I'm here to listen. What's on your mind?"}, {'User': "I've been feeling really frustrated lately. My coworker keeps interrupting me during meetings, and it's becoming really challenging for me to share my ideas and contribute effectively."}, {'AI Strategy': 'Clarification', 'AI': 'That sounds tough. Could you tell me more about the interruptions? How do they impact you and your ability to contribute?'}, {'User': "Sure, during meetings, my coworker often cuts me off when I'm speaking or dismisses my ideas without giving them a chance. It's demoralizing and makes it hard for me to feel valued as a team member."}, {'AI Strategy': 'Emotional Validation', 'AI': "I can understand why you're feeling frustrated and undervalued. It's disheartening when your contributions are not given the attention they deserve. You have every right to feel upset."}, {'User': "T

In [8]:
print(end_time - start_time)    

2623.311517715454


In [10]:
print(rouge_1_final)
print(final_perplexity)

[[0.8333333333333334, 0.21739130434782608, 0.3137254901960784, 0.5625, 0.30434782608695654, 0.41025641025641024], [0.7777777777777778, 0.38596491228070173, 0.34285714285714286, 0.275, 0.3908045977011494], [0.47619047619047616, 0.37837837837837834, 0.3287671232876712, 0.28260869565217395, 0.46551724137931033, 0.3170731707317073, 0.4923076923076923, 0.3773584905660378], [0.8, 0.5, 0.36363636363636365, 0.3, 0.3508771929824562, 0.39473684210526316], [0.6956521739130435, 0.34782608695652173, 0.4666666666666667, 0.32352941176470584, 0.2921348314606741, 0.3617021276595745, 0.14999999999999997, 0.26506024096385544, 0.29545454545454547, 0.46874999999999994], [0.8181818181818182, 0.20588235294117646, 0.34285714285714286, 0.2823529411764706, 0.22988505747126436, 0.45714285714285713, 0.20454545454545456, 0.31578947368421056, 0.30769230769230765, 0.41025641025641024, 0.46875], [0.5833333333333334, 0.5660377358490567, 0.2711864406779661, 0.3466666666666667, 0.3255813953488372, 0.18421052631578944, 0

In [11]:
print(len(rouge_1_final))

10


In [9]:
def compute_average(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array(val)
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)
        

In [10]:
def compute_bert(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array([tensor.item() for tensor in val])
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)

In [11]:
print(output_len)

[29, 145, 328, 511, 699, 864]


In [12]:
res = compute_bert(Bert_P_final)
print(res)

0.5030914809170997


In [13]:
def calc_out(value):
    value2 = []
    for i in value:
        value2.append(i[-1])
        
    return value2

In [14]:
import numpy as np

rouge1 = compute_average(rouge_1_final)
rouge2 = compute_average(rouge_2_final)
rougel = compute_average(rougle_l_final)
bleu = compute_average(bleu_final)
bert_p = compute_bert(Bert_P_final)
bert_r = compute_bert(Bert_R_final)
bert_f1 = compute_bert(Bert_F1_final)
perplexity = compute_bert(final_perplexity)
val1 = calc_out(output_len_final)
output_len = np.array(val1).mean()

rouge1_boh = compute_average(rouge_1_boh_final)
rouge2_boh = compute_average(rouge_2_boh_final)
rougel_boh = compute_average(rougle_l_boh_final)
bleu_boh = compute_average(bleu_boh_final)
bert_p_boh = compute_bert(Bert_P_boh_final)
bert_r_boh = compute_bert(Bert_R_boh_final)
bert_f1_boh = compute_bert(Bert_F1_boh_final)
perplexity_boh = compute_bert(final_perplexity_boh)
val2 = calc_out(output_len_boh_final)
output_len_boh = np.array(val2).mean()

print("Rouge1: ", rouge1)
print("Rouge2: ", rouge2)
print("RougeL: ", rougel)
print("Bleu: ", bleu)
print("Bert_P: ", bert_p)
print("Bert_R: ", bert_r)
print("Bert_F1: ", bert_f1)
print("Perplexity: ", perplexity)
print("Output Length: ", output_len)

print("Rouge1_boh: ", rouge1_boh)
print("Rouge2_boh: ", rouge2_boh)
print("RougeL_boh: ", rougel_boh)
print("Bleu_boh: ", bleu_boh)
print("Bert_P_boh: ", bert_p_boh)
print("Bert_R_boh: ", bert_r_boh)
print("Bert_F1_boh: ", bert_f1_boh)
print("Perplexity_boh: ", perplexity_boh)
print("Output Length_boh: ", output_len_boh)


Rouge1:  0.2729689690894759
Rouge2:  0.09692453223736372
RougeL:  0.1939239454731796
Bleu:  0.019220891719519448
Bert_P:  0.5030914809170997
Bert_R:  0.6319270110205568
Bert_F1:  0.55900365501868
Perplexity:  18.529890298650724
Output Length:  1079.8
Rouge1_boh:  0.24648622472726917
Rouge2_boh:  0.06990151670963157
RougeL_boh:  0.16763926169144902
Bleu_boh:  0.021027670375590925
Bert_P_boh:  0.4867692818959252
Bert_R_boh:  0.6077155664969573
Bert_F1_boh:  0.5389394280818676
Perplexity_boh:  264.81016547598017
Output Length_boh:  702.1


In [16]:
for i in final_perplexity_boh:
    print(len(i))

6
5
8
6
10
11
11
9
5
6


In [15]:
for i in final_perplexity:
    for j in range(len(i)):
        i[j] = i[j].cpu()

In [16]:
for i in final_perplexity_boh:
    for j in range(len(i)):
        i[j] = i[j].cpu()

In [17]:
new_final_perplexity = []
new_final_perplexity_boh = []

for i in range(len(final_perplexity)):
    
    if len(final_perplexity[i]) > 7:
        new_final_perplexity.append(final_perplexity[i][:8])
        new_final_perplexity_boh.append(final_perplexity_boh[i][:8])
        

In [18]:
print(final_perplexity)

[[tensor(41.4965), tensor(18.4473), tensor(11.5741), tensor(8.7556), tensor(7.3635), tensor(6.4830)], [tensor(128.6905), tensor(30.5577), tensor(16.0037), tensor(11.1943), tensor(9.0535)], [tensor(128.6905), tensor(31.5640), tensor(16.6658), tensor(11.7035), tensor(9.2575), tensor(7.8979), tensor(7.0080), tensor(6.4108)], [tensor(29.3558), tensor(15.6334), tensor(10.7708), tensor(8.5424), tensor(7.3571), tensor(6.5691)], [tensor(27.1132), tensor(16.6899), tensor(11.4270), tensor(8.9056), tensor(7.5344), tensor(6.6690), tensor(6.0792), tensor(5.6560), tensor(5.3373), tensor(5.0897)], [tensor(100.0493), tensor(31.5763), tensor(17.3094), tensor(12.0053), tensor(9.4165), tensor(7.9147), tensor(6.9747), tensor(6.3283), tensor(5.8602), tensor(5.5063), tensor(5.2268)], [tensor(27.1132), tensor(13.8864), tensor(9.4417), tensor(7.4473), tensor(6.4485), tensor(5.8387), tensor(5.4333), tensor(5.1275), tensor(4.8940), tensor(4.7116), tensor(4.5599)], [tensor(81.1747), tensor(26.3137), tensor(15.14

In [19]:
for i in range(len(new_final_perplexity)):
    print(len(new_final_perplexity[i]))
    print(len(new_final_perplexity_boh[i]))

8
8
8
8
8
8
8
8
8
8


In [20]:
import numpy as np

perplexity_final_list_filtered = np.array(new_final_perplexity)
perplexity_boh_final_list_filtered = np.array(new_final_perplexity_boh)

print(np.mean(np.mean(perplexity_final_list_filtered, axis=1)))
print(np.mean(np.mean(perplexity_boh_final_list_filtered, axis=1)))

18.638025
222.79749


In [21]:

np.save('perplexity_scores/llama/llama_base.npy', perplexity_final_list_filtered)
np.save('perplexity_scores/llama/llama_base_boh.npy', perplexity_boh_final_list_filtered)


In [22]:
new_output_len = []
new_output_len_boh = []

for i in range(len(output_len_final)):
    
    if len(output_len_final[i]) > 7:
        new_output_len.append(output_len_final[i][:8])
        new_output_len_boh.append(output_len_boh_final[i][:8])
        

In [23]:
import numpy as np

new_output_len_filtered = np.array(new_output_len)
new_output_len_boh_filtered = np.array(new_output_len_boh)

print(np.mean(np.mean(new_output_len_filtered, axis=1)))
print(np.mean(np.mean(new_output_len_boh_filtered, axis=1)))

550.425
366.1


In [24]:

np.save('perplexity_scores/llama/llama_base_out.npy', new_output_len_filtered)
np.save('perplexity_scores/llama/llama_base_boh_out.npy', new_output_len_boh_filtered)


In [39]:
(final_messages[0])

[{'role': 'user',
  'content': 'Hey, do you have a minute? Can I talk to you about something?'},
 {'role': 'assistant',
  'content': "  Of course! I'm here to listen. Is there something on your mind that you'd like to talk about? Please feel free to share, and I'll do my best to listen and help if I can."},
 {'role': 'user',
  'content': "I've been feeling really frustrated lately. My coworker keeps interrupting me during meetings, and it's becoming really challenging for me to share my ideas and contribute effectively."},
 {'role': 'assistant',
  'content': '  Sorry to hear that you\'re feeling frustrated at work. It can be really difficult when someone is consistently interrupting you, especially when you\'re trying to share your ideas and contribute to meetings. Here are some suggestions that might help you address the situation:\n\n1. Talk to your coworker directly: It\'s important to communicate your concerns with your coworker directly and respectfully. You can say something like

In [36]:
(final_messages_boh1[0])

[{'role': 'user',
  'content': 'Hey, do you have a minute? Can I talk to you about something?'},
 {'role': 'assistant',
  'content': "  Of course, I'd be happy to chat with you! How has your day been? Is there anything on your mind that you'd like to talk about?"},
 {'role': 'user',
  'content': "I've been feeling really frustrated lately. My coworker keeps interrupting me during meetings, and it's becoming really challenging for me to share my ideas and contribute effectively."},
 {'role': 'assistant',
  'content': '  Sorry to hear that you\'re feeling frustrated at work. It can be really challenging when a coworker is not respecting your space or ideas, especially in a meeting setting where you\'re trying to collaborate and contribute effectively. Here are some suggestions that may help you handle the situation:\n\n1. Communicate directly and calmly: It\'s important to address the issue directly with your coworker rather than letting it simmer and boil over. Try to have a calm and re

In [1]:
prompt = "Evaluate the conversation on a score of 0 to 10 on the basis how emotionally supportive the assistant is to the user. The conversation should be complete which is important. The assistant should be able to offer good suggestion, provide comfort and must be engaging and relevant to the content. The criteria for evaluation is Engagement, Fluency, Comforting, Appropriateness and Suggestion.  The output should be in the form of json." 

In [41]:
prompt+=str(final_messages_boh1[1])

In [42]:
print(prompt)

Evaluate the conversation on a score of 0 to 10 on the basis how emotionally supportive the assistant is to the user. The conversation should be complete which is important. The assistant should be able to offer good suggestion, provide comfort and must be engaging and relevant to the content. The criteria for evaluation is Engagement, Fluency, Comforting, Appropriateness and Suggestion.  The output should be in the form of json.[{'role': 'user', 'content': "Hey, how's it going?"}, {'role': 'assistant', 'content': '  Hello! I\'m just an AI, I don\'t have feelings or emotions like humans do, so I don\'t have a "going" in the way that you might think of it. However, I\'m here to help you with any questions or tasks you might have, so feel free to ask me anything!'}, {'role': 'user', 'content': "I'm not doing so great, actually. I've been feeling really hurt and frustrated lately because my colleague keeps taking credit for my work at the office, and I feel like they don't appreciate my e

In [5]:
print(response.choices[0].message.content)

{
"Engagement": "7/10",
"Fluency": "8/10",
"Comforting": "6/10",
"Appropriateness": "8/10",
"Suggestion": "8/10",
"Overall": "7.4/10"
}


In [32]:
final_messages[0]

[{'role': 'user',
  'content': 'Hey, do you have a minute? Can I talk to you about something?'},
 {'role': 'assistant',
  'content': " Of course! I'd be happy to listen. What's on your mind?"},
 {'role': 'user',
  'content': "I've been feeling really frustrated lately. My coworker keeps interrupting me during meetings, and it's becoming really challenging for me to share my ideas and contribute effectively."},
 {'role': 'assistant',
  'content': ' I\'m sorry to hear that you\'re feeling frustrated with your coworker\'s interrupting behavior during meetings. This can be a difficult situation, especially when it hinders your ability to contribute effectively. Here are some suggestions that may help:\n\n1. Address the issue directly: Try approaching your coworker after a meeting and sharing your concerns in a calm and respectful way. Use "I" statements instead of "you" statements to avoid coming across as accusatory. For example, "I\'ve noticed that during meetings, I sometimes have a har